In [1]:
from langchain_community.document_loaders import TextLoader # Take raw text of descriptions and convert into format langchain can work with

from langchain_text_splitters import CharacterTextSplitter # Split whole document containing descriptions into meaningful chunks
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

from NLP.Book_Recommendation_Project.LangChain_Top_K import text_splitter

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import pandas as pd
movies = pd.read_csv('movies-extended.csv')

In [4]:
movies

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,Title-And-Plot,Genre-And-Plot,New-Genre
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr...",Kansas Saloon Smashers : A bartender is workin...,"A bartender is working at a saloon, serving dr...",NaN
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov...","Love by the Light of the Moon : The moon, pain...","The moon, painted with a smiling face hangs ov...",NaN
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed...","The Martyred Presidents : The film, just over ...","The film, just over a minute long, is composed...",NaN
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...,"Terrible Teddy, the Grizzly King : Lasting jus...",Lasting just 61 seconds and consisting of two ...,NaN
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...,Jack and the Beanstalk : The earliest known ad...,The earliest known adaptation of the classic f...,NaN
...,...,...,...,...,...,...,...,...,...,...,...
22341,2017,"Hochelaga, Land of Souls (Hochelaga terre des ...",Canadian,François Girard,"Raoul Max Trujillo, Tanaya Beatty, David La Haye",historical drama,"https://en.wikipedia.org/wiki/Hochelaga,_Land_...","One night on the campus of McGill University, ...","Hochelaga, Land of Souls (Hochelaga terre des ...",historical drama : One night on the campus of ...,historical
22342,2017,Indian Horse,Canadian,Stephen Campanelli,"Forrest Goodluck, Michiel Huisman, Michael Mur...",drama,https://en.wikipedia.org/wiki/Indian_Horse_(film),"The Indian Horse family, including six-year-ol...","Indian Horse : The Indian Horse family, includ...","drama : The Indian Horse family, including six...",drama
22343,2017,The Little Girl Who Was Too Fond of Matches (L...,Canadian,Simon Lavoie,NaN,unknown,https://en.wikipedia.org/wiki/The_Little_Girl_...,"In rural 1930s Quebec, Alice lives in house wi...",The Little Girl Who Was Too Fond of Matches (L...,"In rural 1930s Quebec, Alice lives in house wi...",NaN
22344,2017,Meditation Park,Canadian,Mina Shum,"Sandra Oh, Liane Balaban, Don McKellar",drama,https://en.wikipedia.org/wiki/Meditation_Park,"Opened by Mandarin theme song, Meditation Park...",Meditation Park : Opened by Mandarin theme son...,"drama : Opened by Mandarin theme song, Meditat...",drama


In [5]:
movies['movie_id'] = movies.index

In [6]:
movies

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,Title-And-Plot,Genre-And-Plot,New-Genre,movie_id
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr...",Kansas Saloon Smashers : A bartender is workin...,"A bartender is working at a saloon, serving dr...",NaN,0
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov...","Love by the Light of the Moon : The moon, pain...","The moon, painted with a smiling face hangs ov...",NaN,1
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed...","The Martyred Presidents : The film, just over ...","The film, just over a minute long, is composed...",NaN,2
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...,"Terrible Teddy, the Grizzly King : Lasting jus...",Lasting just 61 seconds and consisting of two ...,NaN,3
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...,Jack and the Beanstalk : The earliest known ad...,The earliest known adaptation of the classic f...,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...
22341,2017,"Hochelaga, Land of Souls (Hochelaga terre des ...",Canadian,François Girard,"Raoul Max Trujillo, Tanaya Beatty, David La Haye",historical drama,"https://en.wikipedia.org/wiki/Hochelaga,_Land_...","One night on the campus of McGill University, ...","Hochelaga, Land of Souls (Hochelaga terre des ...",historical drama : One night on the campus of ...,historical,22341
22342,2017,Indian Horse,Canadian,Stephen Campanelli,"Forrest Goodluck, Michiel Huisman, Michael Mur...",drama,https://en.wikipedia.org/wiki/Indian_Horse_(film),"The Indian Horse family, including six-year-ol...","Indian Horse : The Indian Horse family, includ...","drama : The Indian Horse family, including six...",drama,22342
22343,2017,The Little Girl Who Was Too Fond of Matches (L...,Canadian,Simon Lavoie,NaN,unknown,https://en.wikipedia.org/wiki/The_Little_Girl_...,"In rural 1930s Quebec, Alice lives in house wi...",The Little Girl Who Was Too Fond of Matches (L...,"In rural 1930s Quebec, Alice lives in house wi...",NaN,22343
22344,2017,Meditation Park,Canadian,Mina Shum,"Sandra Oh, Liane Balaban, Don McKellar",drama,https://en.wikipedia.org/wiki/Meditation_Park,"Opened by Mandarin theme song, Meditation Park...",Meditation Park : Opened by Mandarin theme son...,"drama : Opened by Mandarin theme song, Meditat...",drama,22344


In [28]:
movies['id-title-plot'] = movies['movie_id'].astype(str) + ' ' + movies['Title'] + ' - ' + movies['Plot']

In [53]:
movies['id-title-plot'] = movies['id-title-plot'].str.replace('\n', ' ', regex=False)

In [54]:
movies

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,Title-And-Plot,Genre-And-Plot,New-Genre,movie_id,id-plot,id-title-plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr...",Kansas Saloon Smashers : A bartender is workin...,"A bartender is working at a saloon, serving dr...",NaN,0,"0 A bartender is working at a saloon, serving ...",0 Kansas Saloon Smashers - A bartender is work...
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov...","Love by the Light of the Moon : The moon, pain...","The moon, painted with a smiling face hangs ov...",NaN,1,"1 The moon, painted with a smiling face hangs ...","1 Love by the Light of the Moon - The moon, pa..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed...","The Martyred Presidents : The film, just over ...","The film, just over a minute long, is composed...",NaN,2,"2 The film, just over a minute long, is compos...","2 The Martyred Presidents - The film, just ove..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...,"Terrible Teddy, the Grizzly King : Lasting jus...",Lasting just 61 seconds and consisting of two ...,NaN,3,3 Lasting just 61 seconds and consisting of tw...,"3 Terrible Teddy, the Grizzly King - Lasting j..."
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...,Jack and the Beanstalk : The earliest known ad...,The earliest known adaptation of the classic f...,NaN,4,4 The earliest known adaptation of the classic...,4 Jack and the Beanstalk - The earliest known ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22341,2017,"Hochelaga, Land of Souls (Hochelaga terre des ...",Canadian,François Girard,"Raoul Max Trujillo, Tanaya Beatty, David La Haye",historical drama,"https://en.wikipedia.org/wiki/Hochelaga,_Land_...","One night on the campus of McGill University, ...","Hochelaga, Land of Souls (Hochelaga terre des ...",historical drama : One night on the campus of ...,historical,22341,22341 One night on the campus of McGill Univer...,"22341 Hochelaga, Land of Souls (Hochelaga terr..."
22342,2017,Indian Horse,Canadian,Stephen Campanelli,"Forrest Goodluck, Michiel Huisman, Michael Mur...",drama,https://en.wikipedia.org/wiki/Indian_Horse_(film),"The Indian Horse family, including six-year-ol...","Indian Horse : The Indian Horse family, includ...","drama : The Indian Horse family, including six...",drama,22342,"22342 The Indian Horse family, including six-y...","22342 Indian Horse - The Indian Horse family, ..."
22343,2017,The Little Girl Who Was Too Fond of Matches (L...,Canadian,Simon Lavoie,NaN,unknown,https://en.wikipedia.org/wiki/The_Little_Girl_...,"In rural 1930s Quebec, Alice lives in house wi...",The Little Girl Who Was Too Fond of Matches (L...,"In rural 1930s Quebec, Alice lives in house wi...",NaN,22343,"22343 In rural 1930s Quebec, Alice lives in ho...",22343 The Little Girl Who Was Too Fond of Matc...
22344,2017,Meditation Park,Canadian,Mina Shum,"Sandra Oh, Liane Balaban, Don McKellar",drama,https://en.wikipedia.org/wiki/Meditation_Park,"Opened by Mandarin theme song, Meditation Park...",Meditation Park : Opened by Mandarin theme son...,"drama : Opened by Mandarin theme song, Meditat...",drama,22344,"22344 Opened by Mandarin theme song, Meditatio...",22344 Meditation Park - Opened by Mandarin the...


In [58]:
movies['id-title-plot'].to_csv('movies_semantic.txt', index=False, header=False)

In [59]:
raw_docs = TextLoader('movies_semantic.txt').load()
text_splitter = CharacterTextSplitter(chunk_size=1500, chunk_overlap=0, separator='\n')
documents = text_splitter.split_documents(raw_docs)

Created a chunk of size 1629, which is longer than the specified 1500
Created a chunk of size 1564, which is longer than the specified 1500
Created a chunk of size 1583, which is longer than the specified 1500
Created a chunk of size 3453, which is longer than the specified 1500
Created a chunk of size 1595, which is longer than the specified 1500
Created a chunk of size 2407, which is longer than the specified 1500
Created a chunk of size 2081, which is longer than the specified 1500
Created a chunk of size 2442, which is longer than the specified 1500
Created a chunk of size 2801, which is longer than the specified 1500
Created a chunk of size 1569, which is longer than the specified 1500
Created a chunk of size 1598, which is longer than the specified 1500
Created a chunk of size 1628, which is longer than the specified 1500
Created a chunk of size 1667, which is longer than the specified 1500
Created a chunk of size 2108, which is longer than the specified 1500
Created a chunk of s

In [60]:
documents[1]

Document(metadata={'source': 'movies_semantic.txt'}, page_content='"3 Terrible Teddy, the Grizzly King - Lasting just 61 seconds and consisting of two shots, the first shot is set in a wood during winter. The actor representing then vice-president Theodore Roosevelt enthusiastically hurries down a hillside towards a tree in the foreground. He falls once, but rights himself and cocks his rifle. Two other men, bearing signs reading ""His Photographer"" and ""His Press Agent"" respectively, follow him into the shot; the photographer sets up his camera. ""Teddy"" aims his rifle upward at the tree and fells what appears to be a common house cat, which he then proceeds to stab. ""Teddy"" holds his prize aloft, and the press agent takes notes. The second shot is taken in a slightly different part of the wood, on a path. ""Teddy"" rides the path on his horse towards the camera and out to the left of the shot, followed closely by the press agent and photographer, still dutifully holding their s

In [61]:
db_movies = Chroma.from_documents(documents, embedding=OpenAIEmbeddings())

In [62]:
query = 'The Shawshank Redemption'
docs = db_movies.similarity_search(query, k=5)

In [63]:
docs

[Document(id='4de2cd2d-84c3-4ddf-b353-e47651f755bd', metadata={'source': 'movies_semantic.txt'}, page_content='"12307 The Shawshank Redemption - In 1947 Portland, Maine, banker Andy Dufresne is convicted of murdering his wife and her lover, and sentenced to two consecutive life sentences at the Shawshank State Penitentiary. He is befriended by contraband smuggler, Ellis ""Red"" Redding, an inmate serving a life sentence. Red procures a rock hammer, and later a large poster of Rita Hayworth for Andy. Working in the prison laundry, Andy is regularly assaulted and raped by ""the Sisters"" and their leader, Bogs.'),
 Document(id='2ae07182-701d-4bed-b0f1-107ca51b7e2d', metadata={'source': 'movies_semantic.txt'}, page_content='The next day at roll call, the guards find Andy\'s cell empty. An irate Norton throws a rock at the poster of Raquel Welch hanging on the cell wall, revealing a tunnel that Andy dug with his rock hammer over the last 19 years. The previous night, Andy escaped through t

In [43]:
def retrieve_semantic_recommendations(query, top_k=10):
    recs = db_movies.similarity_search(query, k=top_k)

    movies_list = []

    for i in range(0, len(recs)):
        raw_text = recs[i].page_content.strip('"').strip()
        first_token, rest = raw_text.split(' ', 1)
        id = int(first_token)
        movies_list.append(id)

    return movies[movies["movie_id"].isin(movies_list)]

In [44]:
retrieve_semantic_recommendations(query, top_k=5)

ValueError: invalid literal for int() with base 10: 'The'